### Parameter Setup

In [ ]:
fabric_tenant = spark.conf.get("spark.fsd.fabric.tenant")


### Load cleaned data from standardization zone

In [ ]:
import pandas as pd
import numpy as np

# Load cleaned Yellow Taxi Trip Records parquet file from standardization zone to pandas dataframe
year = "2022"
standard_path = "03_standard"

pd_df = pd.read_parquet(f"/lakehouse/default/Files/{standard_path}/cleaned_yellow_taxi_tripdata_{year}.parquet", engine="pyarrow")
pd_df.head()


In [ ]:
# Load location zones data from standardization zone
zones_df = pd.read_parquet(f"/lakehouse/default/Files/{standard_path}/nyc_zones.parquet")
zones_df.head()


### Data transformation

In [ ]:
# Derive month columns from pickup and dropoff datetime columns
pd_df["month_pickup"] = pd_df["tpep_pickup_datetime"].dt.month.astype(np.uint8)
pd_df["month_dropoff"] = pd_df["tpep_dropoff_datetime"].dt.month.astype(np.uint8)

# Derive weekday columns
pd_df["weekday_pickup"] = pd_df["tpep_pickup_datetime"].dt.weekday.astype(np.uint8)
pd_df["weekday_dropoff"] = pd_df["tpep_dropoff_datetime"].dt.weekday.astype(np.uint8)

# Derive hour of day columns
pd_df["hour_pickup"] = pd_df["tpep_pickup_datetime"].dt.hour.astype(np.uint8)
pd_df["hour_dropoff"] = pd_df["tpep_dropoff_datetime"].dt.hour.astype(np.uint8)

pd_df.head()


In [ ]:
# Join location zones data with trip data
pd_df = pd_df.join(zones_df.set_index("LocationID"), on="PULocationID")
pd_df.head()


In [ ]:
# Aggregate taxi demand by hour_pickup, weekday_pickup and borough columns
transformed_df = pd_df.groupby(["hour_pickup", "weekday_pickup", "month_pickup", "Borough"]).agg(
    demand=('hour_pickup', 'size'),
    pickup_timestamp=('tpep_pickup_datetime', 'first')
).reset_index()


In [ ]:
# Encoding Borough columns
borough_array = zones_df["Borough"].unique()

transformed_df["borough_id"] = list(map(lambda x: list(borough_array).index(x), transformed_df["Borough"]))


In [ ]:
from sklearn.preprocessing import MinMaxScaler

# Scaling demand column with minmax scaler
scaler = MinMaxScaler()
transformed_df["scaled_demand"] = scaler.fit_transform(transformed_df[["demand"]])
transformed_df


### Sink transformed data to standardization zone

In [ ]:
# Sink transformed trip records to 03_standard path of Fabric OneLake
mssparkutils.fs.mkdirs(f"Files/{standard_path}")

transformed_df.to_csv(f"/lakehouse/default/Files/{standard_path}/transformed_yellow_taxi_tripdata_{year}.csv")


### Register data assets and lineage of target data pipeline to Purview

In [ ]:
%run data_catalog_and_lineage

In [ ]:
%run utils

In [ ]:
purview_data_catalog = PurviewDataCatalog()

fabric_onelake_tenant, fabric_workspace_id, fabric_lakehouse_id = get_onelake_info()
onelake_base_path = f"abfss://{fabric_workspace_id}@{fabric_onelake_tenant}.dfs.fabric.microsoft.com/{fabric_lakehouse_id}/Files"

# Create source data assets list
source_data_assets = []
trip_data_source_file = f"cleaned_yellow_taxi_tripdata_{year}.parquet"
source_data_asset_1 = DataAsset(trip_data_source_file,
                                "parquet",
                                f"{onelake_base_path}/{standard_path}/{trip_data_source_file}")

zones_data_source_file = f"nyc_zones.parquet"                         
source_data_asset_2 = DataAsset(zones_data_source_file,
                                "parquet",
                                f"{onelake_base_path}/{standard_path}/{zones_data_source_file}")
source_data_assets.append(source_data_asset_1)
source_data_assets.append(source_data_asset_2)

# Create sink data assets list
sink_data_assets = []
transformed_trip_data_file = f"transformed_yellow_taxi_tripdata_{year}.csv"
sink_data_asset = DataAsset(transformed_trip_data_file,
                            "parquet",
                            f"{onelake_base_path}/{standard_path}/{transformed_trip_data_file}")
sink_data_assets.append(sink_data_asset)

# Create process data asset
current_notebook_context = mssparkutils.notebook.nb.context
notebook_id = current_notebook_context["currentNotebookId"]
# notebook_name = current_notebook_context["currentNotebookName"]
process_data_asset = DataAsset("data_transformation (Fabric notebook)",
                               "process",
                               f"https://{fabric_tenant}.powerbi.com/groups/{fabric_workspace_id}/synapsenotebooks/{notebook_id}")

# Create lineage for data pipeline
data_pipeline_lineage = DataLineage(source_data_assets, sink_data_assets, process_data_asset)

# Register lineage of data pipeline to Purview
purview_data_catalog.register_lineage(data_pipeline_lineage)
